## MLP Smoothing / Merging Baseline

Train a baseline MLP model for smoothing using graph-level features.

In [58]:
import torch
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

from utils.data_utils import BaseTracksterPairs

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [81]:
transform = T.Compose([
    T.NormalizeFeatures(),
])

# pions do not have negative samples for now
ds = BaseTracksterPairs("data", kind="photon", transform=transform)
ds.data

Data(x=[17844, 4], y=[17844])

In [82]:
# balance the dataset
pos = ds[ds.data.y == 1]
neg = ds[ds.data.y == 0]
len_neg = len(neg)
len_pos = len(pos)
print(f"Positive: {len_pos}, Negative {len_neg}")
shorter = min(len_neg, len_pos)
test_n = int(shorter / 10)

Positive: 7060, Negative 10784


In [112]:
train_set = pos[:shorter - test_n] + neg[:shorter - test_n]
test_set = pos[shorter - test_n:shorter] + neg[shorter - test_n:shorter]
print(f"Train samples: {len(train_set)}, Test samples: {len(test_set)}")
train_dl = DataLoader(train_set, batch_size=16, shuffle=True)
test_dl = DataLoader(test_set, batch_size=16, shuffle=True)

Train samples: 12708, Test samples: 1412


In [113]:
class BaselineMerger(torch.nn.Module):
    def __init__(self, num_inputs, num_hidden=10):
        super(BaselineMerger, self).__init__()

        self.W1 = nn.Linear(num_inputs, num_hidden)
        self.activation = nn.Sigmoid()

        self.W2 = nn.Linear(num_hidden, 1)
        self.output = nn.Sigmoid()

    def forward(self, data):
        x = self.W1(data.x)
        x = self.activation(x)
        x = self.W2(x)
        return self.output(x)


In [114]:
loss_obj = torch.nn.BCELoss()

def train(model, opt, loader):
    epoch_loss = 0
    for batch in loader:
        model.train()
        batch = batch.to(device)
        opt.zero_grad()
        z = model(batch).reshape(-1)
        loss = loss_obj(z, batch.y.type(torch.float))
        epoch_loss += loss
        loss.backward()
        opt.step()
    return float(epoch_loss)

@torch.no_grad()
def test(model, data):
    total = 0
    correct = 0
    for batch in data:
        model.eval()
        batch = batch.to(device)
        z = model(batch).reshape(-1)
        prediction = (z > 0.5).type(torch.int)
        total += len(prediction) 
        correct += sum(prediction == batch.y)
    return (correct / total)

In [116]:
model = BaselineMerger(ds.data.x.shape[1], 16)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
test_acc = test(model, test_dl)
print(f"Initial acc: {test_acc:.4f}")

for epoch in range(30):
    loss = train(model, optimizer, train_dl)
    train_acc = test(model, train_dl)
    test_acc = test(model, test_dl)
    if epoch % 5 == 0:
        print(f'Epoch: {epoch}, loss: {loss:.4f}, train acc: {train_acc:.4f}, test acc: {test_acc:.4f}')

Initial acc: 0.5000
Epoch: 0, loss: 466.5928, train acc: 0.6866, test acc: 0.7096
Epoch: 5, loss: 446.9478, train acc: 0.6953, test acc: 0.7288
Epoch: 10, loss: 445.8517, train acc: 0.6970, test acc: 0.7295
Epoch: 15, loss: 444.9632, train acc: 0.6980, test acc: 0.7316
Epoch: 20, loss: 444.0404, train acc: 0.6981, test acc: 0.7316
Epoch: 25, loss: 442.6979, train acc: 0.6952, test acc: 0.7231


In [117]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score

pred = []
lab = []
for b in test_dl:
    pred += (model(b) > 0.5).type(torch.int).tolist()
    lab += b.y.tolist()

tn, fp, fn, tp = confusion_matrix(lab, pred).ravel()
print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")
print(f'Accuracy: {accuracy_score(lab, pred):.4f}')
print(f'Precision: {precision_score(lab, pred):.4f}')
print(f'Recall: {recall_score(lab, pred):.4f}')

TP: 651, TN: 387, FP: 319, FN: 55
Accuracy: 0.7351
Precision: 0.6711
Recall: 0.9221
